In [1]:
import pandas as pd
import requests
import json
import folium
import IPython.display

def GoogleSearch(restaurantchoice):
    from googleapiclient.discovery import build
    import pprint
    my_api_key = ""
    my_cse_id = "008011913379560691650:bewabl71otw"

    def google_search(search_term, api_key, cse_id, **kwargs):
        service = build("customsearch", "v1", developerKey=api_key)
        res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
        return res['items']

    results = google_search(restaurantchoice, my_api_key, my_cse_id, num=1)
    return results[0]['link']

def RestaurantTopTip(VenueID):
    stats_url = 'https://api.foursquare.com/v2/venues/'+VenueID+'/tips?&client_id='+Client_ID+'&client_secret='+Client_Secret+'&v=20181101'
    statresponse = requests.get(stats_url)
    statdata = statresponse.json()  
    return statdata['response']['tips']['items'][0]['text']


In [2]:
#Foursquare API
Client_ID = 'PXBJ1RUIOUWFUOVE1H1JHH3OCZXZXD0KPA0H3RHVIWHCC1EB'
Client_Secret = ''
iddict = {}
urldict = {}
reslocationdict = {}
location = input("Where would you like to search for: ")
url = 'https://api.foursquare.com/v2/venues/explore?radius=60000&query=food&limit=50&near='+location+'&client_id='+Client_ID+'&client_secret='+Client_Secret+'&v=20181101'
response = requests.get(url)
data = response.json()
centerlocation = (data['response']['geocode']['center']['lat'],data['response']['geocode']['center']['lng'])
for item in data['response']['groups']:
    for value in item['items']:
        print(value['venue']['name'],"><><><>< Type:",value['venue']['categories'][0]['name'])
        reslocationdict[value['venue']['name']] = (value['venue']['location']['lat'],value['venue']['location']['lng'])
        iddict[value['venue']['name']] = value['venue']['id']
        if 'delivery' in value['venue']:
            urldict[value['venue']['name']] = value['venue']['delivery']['url']

Where would you like to search for: Syracuse NY
Dinosaur Bar-B-Que ><><><>< Type: BBQ Joint
Original Grain ><><><>< Type: Restaurant
Varsity Pizza ><><><>< Type: Pizza Place
Pastabilities ><><><>< Type: Italian Restaurant
Funk 'n Waffles ><><><>< Type: Breakfast Spot
Phoebe's Restaurant and Coffee Lounge ><><><>< Type: American Restaurant
Cafe Kubal Downtown Cafe ><><><>< Type: Café
Strong Hearts Cafe ><><><>< Type: Vegetarian / Vegan Restaurant
Alto Cinco ><><><>< Type: Mexican Restaurant
Laci's Tapas Bar ><><><>< Type: Tapas Restaurant
Brooklyn Pickle ><><><>< Type: Sandwich Place
Darwin ><><><>< Type: Sandwich Place
Columbus Bakery And Deli ><><><>< Type: Bakery
Eva's European Sweets ><><><>< Type: Eastern European Restaurant
Francesca's Cucina ><><><>< Type: Italian Restaurant
Liehs & Steigerwald Downtown ><><><>< Type: Deli / Bodega
Picasso's Pastries ><><><>< Type: Bakery
Apizza Regionale ><><><>< Type: Pizza Place
The York ><><><>< Type: Gastropub
Lyncourt Bakery ><><><>< Type: 

In [3]:
while True:
    try:
        restaurantchoice = input("Type the name of the restaurant you want the menu and map for, if you are satisfied type 'Quit' ")
        if restaurantchoice == "Quit" or restaurantchoice == "quit":
            break
        else:
            if restaurantchoice in iddict:
                map = folium.Map(location=centerlocation, zoom_start=13)
                message = restaurantchoice
                marker = folium.Marker(location=reslocationdict[restaurantchoice], 
                    popup=message
                          )
                map.add_child(marker)
                display(map)
                Venue_ID = iddict[restaurantchoice]
                menu_url = 'https://api.foursquare.com/v2/venues/'+Venue_ID+'/menu?&client_id='+Client_ID+'&client_secret='+Client_Secret+'&v=20181101'
                menuresponse = requests.get(menu_url)
                menudata = menuresponse.json()
                if float(menudata['response']['menu']['menus']['count']) == 0:
                    TopTip = RestaurantTopTip(iddict[restaurantchoice])
                    print("Top tip/recommendation:\n",TopTip+"\n")
                    restaurant_url = urldict[restaurantchoice]
                    print("I cant find a menu in my database but here is a URL to their delivery menu!")
                    print(restaurant_url)
                else:
                    menutitle = str(restaurantchoice)+' Menu'
                    menutitleheader = str(menutitle)
                    print(menutitleheader.center(75, "☆"))
                    TopTip = RestaurantTopTip(iddict[restaurantchoice])
                    print("Top tip/recommendation: ",TopTip)
                    for selection in menudata['response']['menu']['menus']['items']:
                        for secondselection in selection['entries']['items']:
                            if 'sectionId' and 'name' and 'entries' and not 'description' in secondselection.keys():
                                print("\n\t\t"+(secondselection['name']).center(75, "-"))
                                for specialty in secondselection['entries']['items']:
                                    if 'price' not in specialty.keys():
                                        print(specialty['name'])
                                    if 'price' in specialty.keys():
                                        print(specialty['name'],"\t\nPrice: $"+specialty['price']+"\n")
                                    if 'description' in specialty.keys():
                                        print(specialty['description']+'\n')
                            elif 'description' in secondselection.keys():
                                print("\n\t\t"+(secondselection['name']).center(75, '-'))
                                print(secondselection['description']+"\n")
                                if 'entries' in secondselection.keys():
                                    for extraitem in secondselection['entries']['items']:
                                        print(extraitem['name'], "\t\nPrice: $"+extraitem['price']+"\n")
                                        if 'description' in extraitem.keys():
                                            print(extraitem['description']+"\n")
            else:
                print("That isn't in your list of choices!")
    except KeyError:
        print("I couldn't find the menu in my database... but I searched the web for you!")
        print("This is my best estimation of the restaurant's website/menu!")
        search = restaurantchoice
        urlsearch = GoogleSearch(search+" "+location+" menu")
        print(urlsearch)
        


Type the name of the restaurant you want the menu and map for, if you are satisfied type 'Quit' Pastabilities


☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆Pastabilities Menu☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆
Top tip/recommendation:  Delicious fresh pasta and bread. I love going by in the morning and smelling the fresh baked bread from across the street where you can stop in and buy some loaves or oil/sauce to take home

		----------------------------------Starters---------------------------------
our bakery fresh stretch bread and one plate of our spicy hot tomato oil are complimentary. an extra plate of spicy hot tomato oil is $3.50

Shrimp Scampi Bruschetta 	
Price: $10.00

On polenta crostini with tomatoes, lemon, shallots

Housemade Fresh Mozzarella Winter Caprese 	
Price: $9.00

With roasted tomatoes, basil, balsamic

Vegetarian West of Utica Greens 	
Price: $8.00

With butter beans, stretch breadcrumbs

Veal Meatballinis 	
Price: $9.00

With house tomato sauce, smothered onions, shaved parm-regg

Classic Calamari Fritti 	
Price: $14.00

With house 50/50 sauce

Garlic Stretch Bread 	
Price: $5.50

With mozzare

In [4]:
link = 'https://health.gov/dietaryguidelines/2015/guidelines/appendix-2/#table-a2-1'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
rs = requests.get(link, headers=header)
dfs = pd.read_html(rs.text)

entry = input("Are you male or female? ")
if entry == 'male':
    maletable = dfs[0]
    maleage = int(input("How old are you? "))
    if maleage < 19:
        age = str(maleage)
    if maleage >= 19:
        age = "19-20"
    if maleage >= 21:
        age = "21-25"
    if maleage >= 26:
        age = "26-30"
    if maleage >= 31:
        age = "31-35"
    if maleage >= 36:
        age = "36-40"
    if maleage >= 41:
        age = "41-45"
    if maleage >= 46:
        age = "46-50"
    if maleage >= 51:
        age = "51-55"
    if maleage >= 56:
        age = "56-60"
    if maleage >= 61:
        age = "61-65"
    if maleage >= 66:
        age = "66-70"
    if maleage >= 71:
        age = "71-75"
    activity = input('How active are you? (Sedentary, Moderately or Active) ')
    if activity == "Sedentary":
        recommendedcal = maletable[maletable['AGE'] == age]['Sedentary[a]']
    if activity == "Moderately":
        recommendedcal = maletable[maletable['AGE'] == age]['Moderatelyactive[b]']
    if activity == "Active":
        recommendedcal = maletable[maletable['AGE'] == age]['Active[c]']
    print("It is recommended that you intake",recommendedcal.values[0],"calories daily based on your age and activity.")
if entry == 'female':
    womantable = dfs[1]
    womanage = int(input("How old are you? "))
    if womanage < 19:
        age = str(womanage)
    if womanage >= 19:
        age = "19-20"
    if womanage >= 21:
        age = "21-25"
    if womanage >= 26:
        age = "26-30"
    if womanage >= 31:
        age = "31-35"
    if womanage >= 36:
        age = "36-40"
    if womanage >= 41:
        age = "41-45"
    if womanage >= 46:
        age = "46-50"
    if womanage >= 51:
        age = "51-55"
    if womanage >= 56:
        age = "56-60"
    if womanage >= 61:
        age = "61-65"
    if womanage >= 66:
        age = "66-70"
    if womanage >= 71:
        age = "71-75"      
    activity = input('How active are you? (Sedentary, Moderately or Active) ')
    if activity == "Sedentary":
        recommendedcal = womantable[womantable['AGE'] == age]['Sedentary[a]']
    if activity == "Moderately":
        recommendedcal = womantable[womantable['AGE'] == age]['Moderatelyactive[b]']
    if activity == "Active":
        recommendedcal = womantable[womantable['AGE'] == age]['Active[c]']
    print("It is recommended that you intake",recommendedcal.values[0],"calories daily based on your age and activity.")

Are you male or female? male
How old are you? 19
How active are you? (Sedentary, Moderately or Active) Moderately
It is recommended that you intake 2800 calories daily based on your age and activity.


In [5]:
def NutrientID(Food):
    site = 'https://ndb.nal.usda.gov/ndb/search/list?SYNCHRONIZER_TOKEN=73fb8253-4c18-4ee8-98b1-cdb7581a5a3e&SYNCHRONIZER_URI=%2Fndb%2Fsearch%2Flist&qt=&qlookup='+Food+'&ds=&manu='
    datas = pd.read_html(site)
    if datas[0]['Food Group or Manufacturer'][0] == 'Fats and Oils':
        ID = datas[0]['NDB Id'][5]
    if datas[0]['Food Group or Manufacturer'][0] == 'Fast Foods':
        ID = datas[0]['NDB Id'][1]
    else:
        ID = datas[0]['NDB Id'][0]
    return ID

totalcal = 0
while True:
    Food = input("Enter what you think you'll order off the menu and what you have already eaten throughout the day: ").replace(" ","+")
    if Food == 'Quit':
        break
    else:
        try:
            _ = int(Food)
        except:
            pass
        else:
            print("You must enter a food item!")
        IDNumber = NutrientID(Food)
        CalorieURL = 'https://api.nal.usda.gov/ndb/V2/reports?&type=f&format=json&api_key='
        foodoptions = {'ndbno' : IDNumber}
        foodresponse = requests.get(CalorieURL, params = foodoptions)
        FoodCalorie = foodresponse.json()
        try:
            for nutrient in FoodCalorie['foods'][0]['food']['nutrients']:
                if nutrient['unit'] == 'kcal':
                    multiplier = input("how many of the "+nutrient['measures'][0]['label']+" are you eating? ")
                    multiplier = int(multiplier)
                    print(int((nutrient['measures'][0]['qty']/nutrient['measures'][0]['qty']))*multiplier,nutrient['measures'][0]['label'],'of',Food.replace("+"," "),'has')
                    print(int(nutrient['measures'][0]['value'])/float(nutrient['measures'][0]['qty'])*multiplier,'calories')
                    calories = int(nutrient['measures'][0]['value'])/float(nutrient['measures'][0]['qty'])*multiplier
                    totalcal = totalcal + calories
                    overcaldif = totalcal - recommendedcal.values[0]
                    if totalcal > recommendedcal.values[0]:
                        print("WAIT! You are over your recommended calorie limit, maybe try to restart and find menu items with less calories?")
                        print("Your total calories would be %.2f!!, you are %.2f calories over!" % (totalcal, overcaldif))
                    else:
                        print("You calorie total is now %.2f" % totalcal)


        except KeyError:
            print("Try to  be more specific... (For example: instead of 'chicken', specify how it is cooked)")




Enter what you think you'll order off the menu and what you have already eaten throughout the day: chicken nuggets
how many of the NUGGETS are you eating? 20
20 NUGGETS of chicken nuggets has
800.0 calories
You calorie total is now 800.00
Enter what you think you'll order off the menu and what you have already eaten throughout the day: Feta Manicotti
how many of the TRAY are you eating? 1
1 TRAY of Feta Manicotti has
509.0 calories
You calorie total is now 1309.00
Enter what you think you'll order off the menu and what you have already eaten throughout the day: chicken sandwich
how many of the SANDWICH are you eating? 1
1 SANDWICH of chicken sandwich has
369.0 calories
You calorie total is now 1678.00
Enter what you think you'll order off the menu and what you have already eaten throughout the day: Vanilla Icecream
how many of the cup are you eating? 2
2 cup of Vanilla Icecream has
840.0 calories
You calorie total is now 2518.00
Enter what you think you'll order off the menu and what y